In [1]:
import pandas as pd
import numpy as np

In [137]:
#Generate file paths
paths={}
for year in range(2010,2018):
    for quarter in range(1,5):
        key='{}Q{}'.format(year,quarter)
        path='{}-house-disburse-detail.csv'.format(key)
        paths[key]=path
for key in ['2009Q3','2009Q4','2018Q1']:   
    path='{}-house-disburse-detail.csv'.format(key)
    paths[key]=path
paths;

In [161]:
all_columns=['BIOGUIDE_ID',
             'OFFICE',
             'QUARTER',#Redundant
             'CATEGORY',
             'DATE',#Redundant
             'PAYEE',
             'START DATE',
             'END DATE',
             'PURPOSE',
             'AMOUNT',
             'YEAR',#Redundant
             'TRANSCODE',#Redundant
             'TRANSCODELONG',#Redundant
             'RECORDID',#Redundant
             'RECIP (orig.)'#Redundant
            ]

## Useful Functions

In [360]:
#Define a function which converts "amounts" to float and returns df.AMOUNT

def amount_numeric_converter(df):
    if df.AMOUNT.dtype =='float64': pass
    else: df.AMOUNT=df.AMOUNT.str.replace(',','').astype('float')
        
    return df.AMOUNT
    

In [ ]:
#Define a function which converts string dates to timestamps
### input = dataframe (df) and date columns to be converted (data_col)
### returns df with timestamp-date columns

def date_converter(df,date_col):
    for col in date_col:
        df[col]=df[col].replace('   ',np.nan).fillna(method='ffill')
        try:
            df[col]=pd.to_datetime(df[col],format='%m/%d/%y')
        except ValueError:
            pass
        #some files have dates in a different string format
        try:
            df[col]=pd.to_datetime(df[col],format='%m/%d/%Y')
        except ValueError:
            print(path)
                 
    return df
    
    

## Question 1

In [363]:
#Define a function which computes the total payments of the data

def total_payment():
    total_payment=0
    for path in paths.values():
        df=pd.read_csv(path,engine='python',usecols=['AMOUNT'])
        payment=amount_numeric_converter(df).sum()
        total_payment += payment
        
    return total_payment

total_payment().round(2)

13660703793.31

## Question 2

In [634]:
#Define a function which computes the STD of coverage period=difference of End/Start date

def std_coverage_period():
    cov_per_data=np.empty((0,))
    for path in paths.values():
        df=pd.read_csv(path,engine='python',usecols=['START DATE','END DATE','AMOUNT']) 
        df['AMOUNT']=amount_numeric_converter(df) 
        df=df[df['AMOUNT']>0].drop(['AMOUNT'],axis=1) #select only data with strictly positive amounts
        df=date_converter(df,['START DATE','END DATE'])
        df['COVERAGE_PERIOD']=(df.iloc[:,1]-df.iloc[:,0]).dt.days #compute time difference of End/Start dates
        cov_per_data=np.append(cov_per_data, df.COVERAGE_PERIOD.get_values())  
    return np.std(cov_per_data)
    

In [635]:
std_coverage_period()

61.407646785702525

## Question 3

In [691]:
#Compute average annual expenditure from 2010-2016

def avg_annual_expenditure():
    annual_expenditures=[]
    for path in paths.values():
        df=pd.read_csv(path,engine='python',usecols=['START DATE','AMOUNT']) 
        
        df=date_converter(df,['START DATE'])
        df['START_YEAR']=df['START DATE'].dt.year

        #select only data from 2010-2016
        df=df[df['START_YEAR']<2017].drop(['START DATE'],axis=1)
        df=df[df['START_YEAR']>2009]

        #select only data with strictly positive amounts
        df['AMOUNT']=amount_numeric_converter(df) 
        df=df[df['AMOUNT']>0] 
        
        #find net amount per year per dataframe
        annual_expenditures.append(df.groupby('START_YEAR').sum())
        
        #compute net annual expenditures
        net_annual_expenditures=pd.concat(annual_expenditures).groupby(level=0).sum()
        
        
    return net_annual_expenditures.mean(axis=0)[0]



In [692]:
avg_annual_expenditure()

1230258512.367134

## Question 4

In [733]:
#Compute expenditures of all offices in 2016
def office_expenditures():
    offices_expenditures=[]
    for path in paths.values():
        df=pd.read_csv(path,engine='python',usecols=['START DATE','AMOUNT','OFFICE']) 

        #select only data from 2016
        df=date_converter(df,['START DATE'])
        df['START_YEAR']=df['START DATE'].dt.year
        df=df[df['START_YEAR']==2016]

        df['AMOUNT']=amount_numeric_converter(df) 

        offices_expenditures.append(df.groupby('OFFICE')['AMOUNT'].sum())
    
    return pd.concat(offices_expenditures).groupby(level=0).sum()

#Find office w/ max total expenditures in 2016
office_expenditures=office_expenditures()
office_expenditures.idxmax()



In [739]:
#Find purpose w/ max total expenditures for office_max_expenditures
def purpose_max_expenditures():
    purpose_expenditures=[]
    for path in paths.values():
        df=pd.read_csv(path,engine='python',usecols=['START DATE','AMOUNT','OFFICE','PURPOSE']) 

        #Select only data for office = 'GOVERNMENT CONTRIBUTIONS'
        df=df[df.OFFICE == 'GOVERNMENT CONTRIBUTIONS'].drop(['OFFICE'],axis=1)
        
        #select only data from 2016
        df=date_converter(df,['START DATE'])
        df['START_YEAR']=df['START DATE'].dt.year
        df=df[df['START_YEAR']==2016]

        df['AMOUNT']=amount_numeric_converter(df) 

        purpose_expenditures.append(df.groupby('PURPOSE')['AMOUNT'].sum())
        
        series_purp_exp=pd.concat(purpose_expenditures).groupby(level=0).sum().sort_values(ascending=False)
    
    return series_purp_exp.index[0],series_purp_exp[0]

#Purpose w/ max total expenditures for office_max_expenditures
purpose_max_expenditures=purpose_max_expenditures()
purpose_max_expenditures[0]

In [743]:
#Fraction of total expenditure
purpose_max_expenditures[1]/(office_expenditures.sum(axis=0))

0.06585479916213956

## Question 5

In [911]:
def staff_salaries():
    list_staff_salaries=[]
    for path in paths.values():
        df=pd.read_csv(path,engine='python',usecols=['BIOGUIDE_ID','START DATE','AMOUNT','PAYEE','CATEGORY'])
        
        #filter data by category ('PERSONNEL COMPENSATION')
        df=df[df.CATEGORY=='PERSONNEL COMPENSATION'].drop(['CATEGORY'],axis=1)
        
        #filter data by year (2016)
        df=date_converter(df,['START DATE'])
        df=df[df['START DATE'].dt.year==2016].drop(['START DATE'],axis=1)
        
        if df.empty: pass
        else:
            #drop NaN
            df=df.dropna(subset=['BIOGUIDE_ID','PAYEE'])

            #Compute payees's salaries from each rep, per df
            df['AMOUNT']=amount_numeric_converter(df)
            df=df.groupby(['BIOGUIDE_ID','PAYEE']).sum()

            list_staff_salaries.append(df)
    
    return list_staff_salaries
        
       
list_staff_salaries=staff_salaries()
list_staff_salaries; 
        
        
    

In [912]:
def max_avg_staff_salary():
    df_total=list_staff_salaries[0]
    for i,df in enumerate(list_staff_salaries[1:]):
        df.columns=['AMOUNT{}'.format(i+1)]
        df_total=pd.merge(df_total,df,left_index=True,right_index=True,how='outer')
    series_salaries=df_total.sum(axis=1).groupby(level=0).mean()
    
    return series_salaries.max()
max_avg_staff_salary()

34755.231785714284

## Question 6

In [1067]:


def reps_4years():
    list_reps=[]
    for quarter,path in paths.items():
        df=pd.read_csv(path,engine='python',usecols=['BIOGUIDE_ID']).dropna()
        reps=df.BIOGUIDE_ID.unique()
        df=pd.DataFrame(data=np.ones(len(reps)),index=reps,columns=[quarter])
        list_reps.append(df)

    df_total=list_reps[0]
    for df in list_reps[1:]:
        df_total=pd.merge(df_total,df,left_index=True,right_index=True,how='outer')

    return df_total.index[df_total.sum(axis=1)>15].tolist()

In [1068]:
reps_4yeas=reps_4years()

In [1027]:
len(df_total.loc[reps_4yrs])

495

In [1010]:
df1=list_reps[0]
df2=list_reps[1]

In [1013]:
pd.merge(df1,df2,left_index=True,right_index=True,how='outer')

,2010Q1,2010Q2
A000014,1.0,1.0
A000022,1.0,1.0
A000055,1.0,1.0
A000210,1.0,1.0
A000358,1.0,1.0
A000361,1.0,1.0
A000362,1.0,1.0
A000363,1.0,1.0
A000364,1.0,1.0
A000365,1.0,1.0


In [1147]:
df=pd.read_csv(paths['2014Q1'],engine='python',usecols=['BIOGUIDE_ID','PAYEE']).dropna()
df1=pd.read_csv(paths['2014Q2'],engine='python',usecols=['BIOGUIDE_ID','PAYEE']).dropna()

        

In [1213]:
df.set_index('BIOGUIDE_ID').drop_duplicates()

,PAYEE
BIOGUIDE_ID,
A000055,UNITED STATES POSTAL SERVICE
A000055,"ABERNATHY, PAMELA M"
A000055,"ARMSTRONG, REBEKAH W"
A000055,"CHRISTENSEN, AUTUMN"
A000055,"DONCHES, MICHELLE M"
A000055,"GROOVER,JENNIFER E"
A000055,"HOUSEL, REUBEN P"
A000055,"KENNEDY,KREG T"
A000055,"LAWSON, CHRISTOPHER L"


In [1151]:
s=df.groupby(['BIOGUIDE_ID','PAYEE']).size()
s1=df1.groupby(['BIOGUIDE_ID','PAYEE']).size()

In [ ]:
df.groupby(['BIOGUIDE_ID','PAYEE']).size().reset_index()

In [1211]:
df.groupby(['BIOGUIDE_ID','PAYEE']).size().reset_index().drop(0,axis=1).groupby('BIOGUIDE_ID').size()

BIOGUIDE_ID
A000055    51
A000210    40
A000361     6
A000367    57
A000369    61
B000013    51
B000213    49
B000287    57
B000490    49
B000574    38
B000589    58
B000755    50
B000911    43
B001149     1
B001227    48
B001234     1
B001242    40
B001243    58
B001244    13
B001245    37
B001248    59
B001250    34
B001251    63
B001252    55
B001255    50
B001256    85
B001257    52
B001259    51
B001260    49
B001262    57
           ..
V000132    49
W000187    39
W000207    36
W000215    38
W000413    52
W000672    51
W000738     1
W000791    50
W000795    60
W000796    48
W000797    40
W000798    58
W000799    68
W000800    66
W000804    40
W000806    44
W000808    55
W000809    72
W000810    39
W000812    47
W000813    48
W000814    52
W000815    47
W000816    52
Y000031    28
Y000033    48
Y000062    52
Y000063    54
Y000064    53
Y000065    53
Length: 458, dtype: int64

In [1158]:
df3=pd.DataFrame(pd.concat([s,s1],axis=1).sum(axis=1),columns=['SIZE'])

In [1194]:
pd.concat([s,s1],axis=1)

0     1
BIOGUIDE_ID PAYEE                                            
A000055     ABERNATHY, PAMELA M                     5.0   1.0
            ACCURATE WORD LLC                       7.0   2.0
            ARMSTRONG, REBEKAH W                    1.0   NaN
            AT & T                                  5.0   4.0
            AT&T                                    6.0   3.0
            AT&T MOBILITY                           5.0   3.0
            BOISE CASCADE                           3.0   1.0
            BSL - GEM LASER EXPRESS INC             NaN   2.0
            CANTEEN REFRESHMENT SERVICES            1.0   1.0
            CAPITOL MARKING PRODUCTS INC            1.0   NaN
            CHARTER COMMUNICATIONS                  6.0   5.0
            CHRISTENSEN, AUTUMN                     1.0   NaN
            CITIBANK GOV CARD SERVICE               5.0   6.0
            CLIFFORD AUBREY LOWRY                   3.0   3.0
            COMCAST                                 1.0   1.0
            COMCAST CABLE                           3.0   2.0
            COMPUTERWORKS                           4.0   3.0
            CULLIGAN OF ELKRIDGE (MARYLAND)         NaN   1.0
            CULLIGAN WATER OF ANNAPOLIS             NaN   2.0
            DAVID L. ANDRUKITUS INC                 2.0   9.0
            DAWSON, MARK E                          1.0   1.0
            DEPT OF HOMELAND SECURITY               3.0   3.0
            DONCHES, MICHELLE M                     2.0   1.0
            EVELYN M STEVEN                         NaN   1.0
            FEDERAL EXPRESS CORP                    3.0   3.0
            FORT PAYNE TIMES JOURNAL                NaN   1.0
            GEM LASER EXPRESS INC                   2.0   2.0
            GROOVER,JENNIFER E                      2.0   1.0
            GSA PUBLIC BUILDING SERVICE             3.0   3.0
            HON. ROBERT ADERHOLT                    6.0   2.0
...                                                 ...   ...
Y000065     HON TED S YOHO                         47.0  47.0
            KAVENEY,BRIAN M                         1.0   1.0
            LEVY COUNTY JOURNAL                     NaN   1.0
            LOCKHEED MARTIN DESKTOP SOLUTIONS INC   1.0   1.0
            MARTIN, WILLIAM C                      12.0  11.0
            MARTIN,WILLIAM C                        2.0   1.0
            MARY ANN DECARLIS TRUST                 3.0   3.0
            MORRIS PUBLISHING GROUP                 1.0   NaN
            NORFLEET, JESSICA P                     6.0   3.0
            NORFLEET,JESSICA P                      3.0   1.0
            PEARSON,KAYLN                           2.0   1.0
            POSOBIEC,BRITTANY P                     2.0   1.0
            PROPIS,RYAN J                           2.0   1.0
            PUBLIC PRINTER                          2.0   1.0
            QUENCH USA LLC                          2.0   NaN
            RASCHID,OMAR A                          2.0   1.0
            RAWSON, GREGORY N                       1.0   3.0
            RAWSON,GREGORY N                        3.0   1.0
            RICHARDSON, DOROTHY G                   NaN   4.0
            RICHARDSON,DOROTHY G                    2.0   1.0
            SAFE TOUCH SECURITY SYSTEMS             4.0   5.0
            TK PROMOTIONS INC                       1.0   NaN
            UNITED PARCEL SERVICE                  10.0  13.0
            UNITED STATES POSTAL SERVICE            3.0   7.0
            UNIVERSITY OF FLORIDA                   2.0   NaN
            VERIZON WIRELESS                        3.0   3.0
            WALSH,JAMES J                           3.0   1.0
            WOODY,CODEYE                            2.0   1.0
            XARISMA INC                             1.0   NaN
            ZENITH REALTY INC                       3.0   3.0

[28909 rows x 2 columns]

In [1160]:
df3=df3[df3.SIZE>=5]

In [1192]:
df3.index=df3.index.droplevel(0)

In [1193]:
df3

,SIZE
PAYEE,
"ABERNATHY, PAMELA M",6.0
ACCURATE WORD LLC,9.0
AT & T,9.0
AT&T,9.0
AT&T MOBILITY,8.0
CHARTER COMMUNICATIONS,11.0
CITIBANK GOV CARD SERVICE,11.0
CLIFFORD AUBREY LOWRY,6.0
COMCAST CABLE,5.0


In [1108]:
df3.iloc[0].get_values().tolist()

[array(['UNITED STATES POSTAL SERVICE', 'ABERNATHY, PAMELA M',
        'ARMSTRONG, REBEKAH W', 'CHRISTENSEN, AUTUMN',
        'DONCHES, MICHELLE M', 'GROOVER,JENNIFER E', 'HOUSEL, REUBEN P',
        'KENNEDY,KREG T', 'LAWSON, CHRISTOPHER L', 'MEDLEY, MEGAN L',
        'MORGAN,JOSEPH', 'RELL,BRIAN E', 'SMITH,CADEANN', 'SWICK,ANNA K',
        'TAYLOR, JENNIFER B', 'TIDWELL,DANIEL', 'WALTERS,TRENT M',
        'WINDHAM, SHEILA', 'RELL, BRIAN E', 'CITIBANK GOV CARD SERVICE',
        'KENNEDY, KREG', 'MORGAN, JOSEPH', 'HON. ROBERT ADERHOLT',
        'CLIFFORD AUBREY LOWRY', 'WALKER COUNTY BD OF EDUCATION',
        'AT&T MOBILITY', 'GSA PUBLIC BUILDING SERVICE',
        'CHARTER COMMUNICATIONS', 'COMCAST CABLE', 'VERIZON WIRELESS',
        'FEDERAL EXPRESS CORP', 'AT&T', 'AT & T', 'ACCURATE WORD LLC',
        'PUBLIC PRINTER', 'DAVID L. ANDRUKITUS INC', 'COMPUTERWORKS',
        'ICONSTITUENT LLC', 'DEPT OF HOMELAND SECURITY',
        'USW HOLDING CO LLC', 'KYUKA WATERS INC',
        'CANTEEN 

In [1053]:
len(df.set_index('BIOGUIDE_ID').loc[reps_4yrs.tolist()])
#[reps_4yrs.tolist()]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


51846

In [1089]:
{1,2,3}.union({1,4})

{1, 2, 3, 4}

In [1135]:
d={'A':[1,2,3]}
d['A']

[1, 2, 3]

In [1136]:
l=d['A']
l.extend(['a'])
d['A']=l

In [1137]:
d

{'A': [1, 2, 3, 'a']}

In [1132]:
l=[1, 2, 3]
l.extend([5])

In [1133]:
l

[1, 2, 3, 5]